In this optimization, I removed one more column, kept my binning, adjusted the nodes and neurons, then I adjusted the batch size(I made it bigger) to help train the model

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
df = application_df.drop(["EIN", "NAME", "APPLICATION_TYPE", "SPECIAL_CONSIDERATIONS", "STATUS", "ORGANIZATION", "USE_CASE", "CLASSIFICATION"], axis=1)
df.head(5)

,AFFILIATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Independent,0,5000,1
1,Independent,1-9999,108590,1
2,CompanySponsored,0,5000,0
3,CompanySponsored,10000-24999,6692,1
4,Independent,100000-499999,142590,1


In [4]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_counts = df['AFFILIATION'].value_counts()
application_types_to_replace = application_counts[application_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['AFFILIATION'] = df['AFFILIATION'].replace(app,"Other")

# Check to make sure binning was successful
df['AFFILIATION'].value_counts()


Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 1000
application_counts = df['INCOME_AMT'].value_counts()
application_types_to_replace = application_counts[application_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['INCOME_AMT'] = df['INCOME_AMT'].replace(app,"Other")

# Check to make sure binning was successful
df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
Other             2790
Name: INCOME_AMT, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_counts = df['ASK_AMT'].value_counts()
application_types_to_replace = application_counts[application_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['ASK_AMT'] = df['ASK_AMT'].replace(app,"Other")

# Check to make sure binning was successful
df['ASK_AMT'].value_counts()

5000     25398
Other     8901
Name: ASK_AMT, dtype: int64

In [7]:

categorical_columns = ['AFFILIATION',
                       'INCOME_AMT', 'ASK_AMT']
df_encoded = pd.get_dummies(df, columns=categorical_columns)
df_encoded.head()


,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999,INCOME_AMT_Other,ASK_AMT_5000,ASK_AMT_Other
0,1,0,1,0,1,0,0,0,1,0
1,1,0,1,0,0,0,0,1,0,1
2,0,1,0,0,1,0,0,0,1,0
3,1,1,0,0,0,0,0,1,0,1
4,1,0,1,0,0,1,0,0,0,1


In [8]:

from keras.src.feature_column import dense_features_v2
# Split our preprocessed data into our features and target arrays
y = df_encoded['IS_SUCCESSFUL']

X = df_encoded.drop(columns = "IS_SUCCESSFUL")


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:

input_features = X_train_scaled.shape[1]
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation='relu', input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 50        
                                                                 
 dense_1 (Dense)             (None, 50)                300       
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 50)                2550      
                                                                 
 dense_4 (Dense)             (None, 50)                2550      
                                                                 
 dense_5 (Dense)             (None, 50)                2550      
                                                                 
 dense_6 (Dense)             (None, 50)               

In [12]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
nn.fit(X_train_scaled, y_train, epochs=100, batch_size=64, validation_data=(X_test_scaled, y_test))

Epoch 1/100
429/429 [==============================] - 4s 4ms/step - loss: 0.6250 - accuracy: 0.6833 - val_loss: 0.6230 - val_accuracy: 0.6729
Epoch 2/100
429/429 [==============================] - 2s 5ms/step - loss: 0.6189 - accuracy: 0.6871 - val_loss: 0.6226 - val_accuracy: 0.6813
Epoch 3/100
429/429 [==============================] - 2s 4ms/step - loss: 0.6176 - accuracy: 0.6876 - val_loss: 0.6215 - val_accuracy: 0.6816
Epoch 4/100
429/429 [==============================] - 2s 4ms/step - loss: 0.6175 - accuracy: 0.6869 - val_loss: 0.6218 - val_accuracy: 0.6816
Epoch 5/100
429/429 [==============================] - 2s 4ms/step - loss: 0.6171 - accuracy: 0.6869 - val_loss: 0.6210 - val_accuracy: 0.6816
Epoch 6/100
429/429 [==============================] - 2s 4ms/step - loss: 0.6163 - accuracy: 0.6877 - val_loss: 0.6208 - val_accuracy: 0.6816
Epoch 7/100
429/429 [==============================] - 1s 3ms/step - loss: 0.6158 - accuracy: 0.6875 - val_loss: 0.6208 - val_accuracy: 0.6813

In [14]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.6212 - accuracy: 0.6818 - 473ms/epoch - 2ms/step
Loss: 0.6212390661239624, Accuracy: 0.6817784309387207
